# "策略"模式
## 经典"策略"模式

三部分组成：
- 上下文 order
- 策略 Promotion
- 具体策略 FidelityPromo BulkItemPromo LargeOrderPromo


In [6]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer','name fidelity')

class LineItem:
    '''每种产品信息，产品名，数量，单价等'''
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.quantity * self.price


class Order:
    '''账单'''
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fm = '<Order total:{:.2f} due:{:.2f}>'
        return fm.format(self.total(),self.due())

class Promotion(ABC):
    '''策略，抽象基类'''
    @abstractmethod
    def discount(self, order):
        '''返回折扣金额（正值）'''

class FidelityPromo(Promotion):
    '''满1000积分提供5%的折扣'''
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000.0 else order.total()

class BulkItemPromo(Promotion):
    '''单个商品数量为20或以上时提供10%的折扣'''
    def discount(self, order):
        d = 0
        for item in order.cart:
            if item.quantity >= 20 :
                d += item.total() * 0.1
        return d

class LargeOrderPromo(Promotion):
    '''订单中达到或超过10种商品时提供7%的折扣'''
    def discount(self, order):
        distinct_item = { item.product for item in order.cart } #使用了集，集内不能有重复元素
        if len(distinct_item) >= 10 :
            return order.total() * 0.07
        return 0


joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, 5),
        LineItem('apple', 10,1.5),
        LineItem('watermelon', 5, 5.0)]

Order(joe, cart, FidelityPromo)

TypeError: discount() missing 1 required positional argument: 'order'